In [1]:
%matplotlib inline
import numpy as np
from sklearn.preprocessing import LabelEncoder
import mne
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, f1_score
from sklearn.feature_selection import RFECV
from mne.preprocessing import ICA
from sklearn.model_selection import StratifiedKFold
from sklearn.ensemble import RandomForestClassifier

In [2]:
data = np.load("../../datasets/beta/data.npy")
data.shape

(160, 64, 1000)

In [3]:
# criação de um objeto "info"
n_channels = 64
sfreq = 250
ch_names = list(np.load("../../datasets/beta/channels.npy"))
ch_types = ['eeg'] * n_channels
info = mne.create_info(ch_names, sfreq=sfreq, ch_types=ch_types)

In [4]:
labels = np.load("../../datasets/beta/labels.npy")
print(labels, labels.shape)

[ 8.6  8.8  9.   9.2  9.4  9.6  9.8 10.  10.2 10.4 10.6 10.8 11.  11.2
 11.4 11.6 11.8 12.  12.2 12.4 12.6 12.8 13.  13.2 13.4 13.6 13.8 14.
 14.2 14.4 14.6 14.8 15.  15.2 15.4 15.6 15.8  8.   8.2  8.4  8.6  8.8
  9.   9.2  9.4  9.6  9.8 10.  10.2 10.4 10.6 10.8 11.  11.2 11.4 11.6
 11.8 12.  12.2 12.4 12.6 12.8 13.  13.2 13.4 13.6 13.8 14.  14.2 14.4
 14.6 14.8 15.  15.2 15.4 15.6 15.8  8.   8.2  8.4  8.6  8.8  9.   9.2
  9.4  9.6  9.8 10.  10.2 10.4 10.6 10.8 11.  11.2 11.4 11.6 11.8 12.
 12.2 12.4 12.6 12.8 13.  13.2 13.4 13.6 13.8 14.  14.2 14.4 14.6 14.8
 15.  15.2 15.4 15.6 15.8  8.   8.2  8.4  8.6  8.8  9.   9.2  9.4  9.6
  9.8 10.  10.2 10.4 10.6 10.8 11.  11.2 11.4 11.6 11.8 12.  12.2 12.4
 12.6 12.8 13.  13.2 13.4 13.6 13.8 14.  14.2 14.4 14.6 14.8 15.  15.2
 15.4 15.6 15.8  8.   8.2  8.4] (160,)


In [5]:
# método para transformar labels categóricos
data_correct = data
le = LabelEncoder()
events = np.column_stack((
    np.array(range(len(labels))),
    np.zeros(160, dtype=int),
    le.fit_transform(labels))
)

event_dict = {str(value): index  for index, value in enumerate(sorted(set(labels)))}
mne_data = mne.EpochsArray(data_correct, info, events, event_id=event_dict)
filtered_mne_data = mne_data.filter(6, 18)
filtered_mne_data

Not setting metadata
160 matching events found
No baseline correction applied
0 projection items activated
Setting up band-pass filter from 6 - 18 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 6.00
- Lower transition bandwidth: 2.00 Hz (-6 dB cutoff frequency: 5.00 Hz)
- Upper passband edge: 18.00 Hz
- Upper transition bandwidth: 4.50 Hz (-6 dB cutoff frequency: 20.25 Hz)
- Filter length: 413 samples (1.652 s)



[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done 10240 out of 10240 | elapsed:    0.7s finished


Number of events,160
Events,10.0: 410.2: 410.4: 410.6: 410.8: 411.0: 411.2: 411.4: 411.6: 411.8: 412.0: 412.2: 412.4: 412.600000000000001: 412.8: 413.0: 413.200000000000001: 413.4: 413.600000000000001: 413.8: 414.0: 414.200000000000001: 414.4: 414.600000000000001: 414.8: 415.0: 415.200000000000001: 415.4: 415.600000000000001: 415.8: 48.0: 48.2: 48.4: 48.6: 48.799999999999999: 49.0: 49.2: 49.4: 49.6: 49.8: 4
Time range,0.000 – 3.996 s
Baseline,off


In [6]:
# Estimando o ruído de fundo

# intervalos de tempo sem estímulo (0 a 0,5 segundos e 2,5 a 3 segundos)
base_start = 0
base_end = 125
rest_start = 625
rest_end = 750

# armazena uma lista com as médias de potência para cada canal
noise_power = []
# consideramos a primeira amostra (1º target, 1º trial)
for channel_data in data[0, :, :]:
    fft_result = np.fft.fft(channel_data)
    # densidade espectral de potência (PSD)
    psd = np.abs(fft_result) ** 2
    # média da potência nos intervalos de tempo sem estímulo
    base_power = np.mean(psd[base_start:base_end])
    rest_power = np.mean(psd[rest_start:rest_end])
    # média das duas médias de potência obtidas anteriormente
    mean_noise_power = (base_power + rest_power) / 2
    noise_power.append(mean_noise_power)
# média das médias de potência de todos os canais para estimar o ruído de fundo
mean_noise_power = np.mean(noise_power)
mean_noise_power

16196.549880349

In [7]:
sr = 250

# frequências alvo
target_frequencies = np.arange(8, 16, 0.2)
# lista para armazenar as amplitudes nas frequências alvo
target_amplitudes = []

for channel_data in mne_data.get_data():
    target = []
    for electrode in channel_data:

        fft_result = np.fft.fft(electrode)
        psd = np.abs(fft_result) ** 2
        frequencies = np.fft.fftfreq(len(fft_result), 1 / sr)
        target_amplitudes_trial = []
        for target_frequency in target_frequencies:
            # encontrando o índice da frequência alvo no espectro de frequência
            index = np.argmin(np.abs(frequencies - target_frequency))
            # amplitude na frequência alvo
            amplitude = np.sqrt(psd[index])
            target_amplitudes_trial.append(amplitude)

        target.append(target_amplitudes_trial)


    target_amplitudes.append(target)

target_amplitudes = np.array(target_amplitudes)
target_amplitudes.shape

(160, 64, 40)

In [8]:
# forçando (estragando) valor de "estimated_background_noise" para não sobrar valores negativos
target_amplitudes_adjusted = np.abs(target_amplitudes - mean_noise_power)

# subtraindo o ruído de fundo das amplitudes
narrow_band_SNR = 10 * np.log10(target_amplitudes_adjusted / mean_noise_power)
# print(narrow_band_SNR.shape)

total_power = np.sum(target_amplitudes_adjusted)
wide_band_SNR = 10 * np.log10(target_amplitudes_adjusted / total_power)

print(wide_band_SNR.shape)

data = np.array([narrow_band_SNR, wide_band_SNR])
data = data.swapaxes(0, 1)
data = data.swapaxes(1, 2)
data = data.reshape(data.shape[0], data.shape[1], data.shape[2] * data.shape[3])
print(data.shape)

(160, 64, 40)
(160, 64, 80)


# Primeira alternativa (remoção manual de características)

In [9]:
ch_names = np.load("../../datasets/beta/channels.npy")

channels_to_keep = ['PZ', 'PO3', 'PO5', 'PO4', 'PO6', 'POZ', 'O1', 'OZ', 'O2']
indexes_to_remove = np.where(np.isin(ch_names, channels_to_keep, invert=True))

print("Before", data.shape)
data_filtered = np.delete(data, indexes_to_remove, axis=1)
print("After", data_filtered.shape)
print("Labels", labels.shape)

Before (160, 64, 80)
After (160, 9, 80)
Labels (160,)


In [10]:
data_filtered_reshape = data_filtered.reshape(data_filtered.shape[0], data_filtered.shape[1] * data_filtered.shape[2])
print(data_filtered_reshape.shape)

(160, 720)


In [11]:
X = StandardScaler().fit_transform(data_filtered_reshape)
y = LabelEncoder().fit_transform(labels)

x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

clf = SVC(kernel='linear', C=1, random_state=42, probability=True)
clf.fit(x_train, y_train)
y_pred = clf.predict(x_test)

print("accuracy", accuracy_score(y_test, y_pred))
print("f1_score", f1_score(y_test, y_pred, average="weighted"))

accuracy 0.041666666666666664
f1_score 0.041666666666666664


# Segunda alternativa (utilizando remoção automático de características RFE)

In [21]:
# reorganizando os dados

X_reshape_auto = data.reshape(data.shape[0], data.shape[1] * data.shape[2])
print(X_reshape_auto.shape)

(160, 5120)


In [22]:
X_reshape_auto = StandardScaler().fit_transform(X_reshape_auto)

rfe = RFECV(SVC(kernel="linear"), step=0.0001, min_features_to_select=1, cv=3)
X_final = rfe.fit_transform(X_reshape_auto, y)
print(X_final.shape)

(160, 742)


In [23]:
np.save(f"../../datasets/beta/X_RFE_SVM_linear.npy", X_final)

In [24]:
x_train, x_test, y_train, y_test = train_test_split(X_final, y, test_size=0.3, random_state=42)

clf = SVC(kernel='linear', C=1, random_state=42, probability=True)
clf.fit(x_train, y_train)
y_pred = clf.predict(x_test)  

print("accuracy", accuracy_score(y_test, y_pred))
print("f1_score", f1_score(y_test, y_pred, average="weighted"))

accuracy 0.1875
f1_score 0.15525793650793648


# Usando Random Forest

In [12]:
# reorganizando os dados

X_reshape_auto = data.reshape(data.shape[0], data.shape[1] * data.shape[2])
print(X_reshape_auto.shape)

(160, 5120)


In [ ]:
X_reshape_auto = StandardScaler().fit_transform(X_reshape_auto)

rfe = RFECV(RandomForestClassifier(), step=0.0001, min_features_to_select=1, cv=3)
X_final = rfe.fit_transform(X_reshape_auto, y)
print(X_final.shape)

In [49]:
x_train, x_test, y_train, y_test = train_test_split(X_final, y, test_size=0.3, random_state=42)

# Criar um modelo Random Forest
clf = RandomForestClassifier(n_estimators=100, random_state=42)

# Treinar o modelo no conjunto de treinamento
clf.fit(x_train, y_train)

# Fazer previsões no conjunto de teste
y_pred = clf.predict(x_test)

# Avaliar o desempenho do modelo
print("Accuracy:", accuracy_score(y_test, y_pred))
print("F1 Score (weighted):", f1_score(y_test, y_pred, average="weighted"))

Accuracy: 0.041666666666666664
F1 Score (weighted): 0.036111111111111115
